In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights

In [2]:
model = models.vit_b_16(weights="ViT_B_16_Weights.IMAGENET1K_V1")

In [3]:
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from collections import OrderedDict

In [4]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 10))]))
model = model.to(device)

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [5]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.1315, -0.2542,  0.4239,  0.6746,  0.0775,  0.4267, -0.4539,  0.4210,
          0.2361,  0.3482]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [6]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune_3'), None)

Epoch 10 done, train loss: 0.0009 val acc: 1.0000
Epoch 20 done, train loss: 0.0002 val acc: 1.0000
Epoch 30 done, train loss: 0.0001 val acc: 1.0000
Epoch 40 done, train loss: 0.0000 val acc: 1.0000
Epoch 50 done, train loss: 0.0000 val acc: 1.0000
Epoch 60 done, train loss: 0.0000 val acc: 1.0000
Epoch 70 done, train loss: 0.0000 val acc: 1.0000
Epoch 80 done, train loss: 0.0000 val acc: 1.0000
Epoch 90 done, train loss: 0.0000 val acc: 1.0000
Epoch 100 done, train loss: 0.0000 val acc: 1.0000


In [7]:
model = torch.load(os.path.join('..', 'models', 'finetune_3', 'digit-model.pt'))

In [8]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [9]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0
